In [ ]:
!pip install ollama
import ollama
import json
from typing import Dict, List

# --- Step 1: Control parameters and generate a response ---
# The `generate` function sends a request to the Ollama server.
# You can control model parameters through the `options` dictionary.
# A full list of parameters can be found in the Ollama documentation.

# Define the model and prompt
model_name = 'llama3.1'

# Format prompt for Llama3.1
def format_messages(messages: List[Dict[str, str]]) -> List[str]:
    """Format messages for Llama-3 chat models.
    
    The model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and 
    alternating (u/a/u/a/u...). The last message must be from 'user'.
    """
    prompt: List[str] = []
    # print(messages[0]['role'])
    if messages[0]["role"] == "system":
        content = "".join(["<|start_header_id|>system<|end_header_id|>\n\n", messages[0]["content"], "<|eot_id|>", "<|start_header_id|>user<|end_header_id|>\n\n",messages[1]["content"],"<|eot_id|>"])
        messages = [{"role": messages[1]["role"], "content": content}] + messages[2:]

    for user, answer in zip(messages[::2], messages[1::2]):
        prompt.extend(["<|start_header_id|>user<|end_header_id|>", "\n\n", (user["content"]).strip(), " <|eot_id|>"])

    prompt.extend(["<|begin_of_text|>", (messages[0]["content"]).strip(), "<|start_header_id|>assistant<|end_header_id|>\n\n"])

    return "".join(prompt)

# Define the generation parameters
params = {
    "top_p": 0.6,
    "temperature": 0.9,
    "top_k": 50,
    "seed": 7103,
    "max_new_tokens": 8192
}

# Generate the response
print(f"Generating a response with the following parameters: {params}\n")
response = ollama.generate(
    model=model_name,
    prompt=prompt_text,
    options=params,
    stream=False, # Set to True for streaming responses
    format="json" # Forces the response to be in JSON format
)

# The generated text is in the 'response' key
generated_text = response['response']
print(f"Generated Story:\n{generated_text}\n")


# --- Step 2: Save the answer to a file ---
# You can save the response and other metadata to a file,
# for example, a JSON file to keep the data structured.

# Create a dictionary to hold the data
data_to_save = {
    'model': model_name,
    'prompt': prompt_text,
    'parameters': params,
    'response_text': generated_text
}

# Define a filename
filename = 'ollama_story_output.json'

# Save the data to the JSON file
with open(filename, 'w') as f:
    json.dump(data_to_save, f, indent=4)

print(f"Response and parameters saved to '{filename}'.")